In [1]:
import pandas as pd
import json
import ast

In [8]:
parallel_dataset_bruk_dataset = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/bruk_parallel.csv")
parallel_dataset_ng_dataset = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/ng_parallel.csv")

In [9]:
def format_ann(ann_str):
    try:
        ann_str = ann_str.replace("'", '"')
        ann_str = ann_str.replace("–", '-')
        json_ann = ast.literal_eval(ann_str)
        annotations = set(json_ann.values())
        return annotations
    
    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error parsing annotation: {e}")
        return None
    
def format_df(df):
    df['ann'] = df.apply(lambda x: format_ann(x['orig_ann']), axis=1)
    sentence_and_ann = df[['original_sentence', 'changed_sentence', 'ann']]
    return sentence_and_ann

In [13]:
bruk_sentence_and_ann_df = format_df(parallel_dataset_bruk_dataset)
ng_sentence_and_ann_df = format_df(parallel_dataset_ng_dataset)

In [14]:
ng_sentence_and_ann_df.head()

,original_sentence,changed_sentence,ann
0,"Слідчі встановили, що у 2018-2019 рр. управлін...","Слідчі встановили, що у 2018-2019 рр. управлін...","{(Слідчі, JOB)}"
1,Суд надав слідчим доступ до документації щодо ...,Суд надав слідчій доступ до документації щодо ...,"{(слідчим, JOB)}"
2,"Директором та засновником ТОВ «Армшляхбуд», ст...","Директоркою та засновницею ТОВ «Армшляхбуд», с...","{(Директором, JOB), (засновником, JOB)}"
3,Співвласником цієї компанії у свою чергу з час...,Співвласницею цієї компанії у свою чергу з час...,"{(голова наглядової ради, JOB)}"
4,Одесит Іван Плачков почав працювати в компанії...,Одеситка Іванна Плачкова почала працювати в ко...,"{(головою правління, JOB)}"


In [ ]:
len(bruk_sentence_and_ann_df)

492

In [12]:
len(ng_sentence_and_ann_df)

1021

In [ ]:
parallel_df = pd.concat([ng_sentence_and_ann_df, bruk_sentence_and_ann_df])

In [19]:
parallel_df.reset_index(drop=True, inplace=True)

In [21]:
parallel_df.head()

,original_sentence,changed_sentence,ann
0,"Слідчі встановили, що у 2018-2019 рр. управлін...","Слідчі встановили, що у 2018-2019 рр. управлін...","{(Слідчі, JOB)}"
1,Суд надав слідчим доступ до документації щодо ...,Суд надав слідчій доступ до документації щодо ...,"{(слідчим, JOB)}"
2,"Директором та засновником ТОВ «Армшляхбуд», ст...","Директоркою та засновницею ТОВ «Армшляхбуд», с...","{(Директором, JOB), (засновником, JOB)}"
3,Співвласником цієї компанії у свою чергу з час...,Співвласницею цієї компанії у свою чергу з час...,"{(голова наглядової ради, JOB)}"
4,Одесит Іван Плачков почав працювати в компанії...,Одеситка Іванна Плачкова почала працювати в ко...,"{(головою правління, JOB)}"


In [ ]:
# parallel_df.to_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/parallel_df.csv", index=0)